In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue.core.sheet import Sheet
from tyssue.config.json_parser import load_default

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.dynamics.apoptosis_model import SheetApoptosisModel as model
from tyssue.dynamics.sheet_vertex_model import SheetModel as basemodel
from tyssue.solvers.sheet_vertex_solver import Solver as solver

#from tyssue.dynamics.sheet_isotropic_model import isotropic_relax

from tyssue.draw.vispy_draw import vp_view
from tyssue.topology.sheet_topology import type1_transition, remove_face

from tyssue.draw.mpl_draw import sheet_view
import tyssue.draw.mpl_draw as draw
from tyssue.stores import load_datasets

In [2]:

specs = {
    'edge': {
        'dx': 0.0, 'dy': 0.0, 'dz': 0.0,
        'length': 0.0,
        'nx': 0.0, 'ny': 0.0, 'nz': 1.0,
        'srce': 0, 'trgt': 0, 'face': 0, 'cell': 0,
        'is_apical': True,
        'is_basal': False,
        'sub_area': 0.0,
        'line_tension': 0.14,
        'basal_elasticity': 0.0,
        },
    'face': {
        'area': 1.0,
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        'contractility': 0.0,
        },
    'cell': {
        'vol': 1.0,
        'vol_elasticity': 0.0, 
        'is_alive': True,
        'num_sides': 6,
        'perimeter': 1.0,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        },
     'settings': {
        'geometry': 'cylindrical',
        'height_axis': 'z'
        },
     'vert': {
        'is_active': True,
        'is_basal': False,
        'x': 0.0, 'y': 0.0, 'z': 0.0,
        }
    }




In [3]:
def get_opposites(edge_df):

    st_indexed = edge_df[['srce', 'trgt']].reset_index().set_index(
        ['srce', 'trgt'], drop=False)
    flipped = st_indexed.index.swaplevel(0, 1)
    flipped.names = ['srce', 'trgt']
    opposite = st_indexed.loc[flipped, 'edge'].values
    return opposite

In [16]:
h5store = 'small_hexagonal.hf5'
datasets = load_datasets(h5store,
                         data_names=['face', 'vert', 'edge'])


coords = ['x', 'y', 'z']
datasets['cell'] = pd.DataFrame(index=datasets['face'].index,
                                columns=coords)
datasets['cell'].index.name = 'cell'

datasets['cell']['z'] = datasets['face']['z']

theta = np.arctan2(datasets['face']['y'],
                   datasets['face']['x'])

rho = np.hypot(datasets['face']['y'],
               datasets['face']['x'])

datasets['cell']['x'] = 2*rho/3 * np.cos(theta)
datasets['cell']['y'] = 2*rho/3 * np.sin(theta)

n_verts = datasets['vert'].shape[0]

basal_idx = pd.Index(np.arange(datasets['face'].shape[0])+n_verts,
                     name='vert')
basal_verts = pd.DataFrame(index=basal_idx, 
                           columns=datasets['vert'].columns)

datasets['vert'] = pd.concat([datasets['vert'], basal_verts])
datasets['vert'].loc[basal_idx, 'z'] = datasets['face']['z'].values
datasets['vert'].loc[basal_idx, 'x'] = rho.values/3 * np.cos(theta.values)
datasets['vert'].loc[basal_idx, 'y'] = rho.values/3 * np.sin(theta.values)
datasets['vert'].loc[basal_idx, 'is_active'] = 1


datasets['edge']['cell'] = datasets['edge']['face']
datasets['edge']['is_basal'] = 0

opposite = get_opposites(datasets['edge'])
opp_face = datasets['edge'].loc[opposite, 'face'].values
opp_cell = datasets['edge'].loc[opposite, 'cell'].values

basal_edge_idx = pd.Index(
    np.arange(datasets['edge'].shape[0]) +
    datasets['edge'].index[-1], name='edge')

basal_edges = pd.DataFrame(index=basal_edge_idx,
                           columns=['srce', 'trgt', 'is_basal'])

basal_edges['srce'] = datasets['edge']['cell'].values + n_verts
basal_edges['trgt'] = opp_cell + n_verts

datasets['edge'] = pd.concat([datasets['edge'], basal_edges])




In [17]:
from tyssue.core.objects import Epithelium


In [18]:
eptm = Epithelium('mixed', datasets, specs)

In [29]:
from tyssue.geometry.sheet_geometry import SheetGeometry

class MixedGeometry(SheetGeometry):
    
    
    @classmethod
    def update_all(cls, eptm):
        '''
        Updates the epithelium geometry by updating:
        * the edge vector coordinates
        * the edge lengths
        * the normals to each edge associated face
        * the face areas
        * the vertices heights (depends on geometry)
        * the face volumes (depends on geometry)

        '''
        cls.update_dcoords(eptm)
        cls.update_length(eptm)
        cls.update_height(eptm)
        cls.update_normals(eptm)
        cls.update_areas(eptm)
        cls.update_perimeters(eptm)
        cls.update_vol(eptm)
    
    @staticmethod
    def update_vol(eptm):
        
        face_pos = eptm.upcast_face(
            eptm.face_df[eptm.coords])
        cell_pos = eptm.upcast_cell(
            eptm.cell_df[eptm.coords])
        
        eptm.edge_df['sub_vol'] = np.sum(
            (face_pos - cell_pos) *
            eptm.edge_df[eptm.ncoords].values, axis=1) / 6
        eptm.edge_df['sub_vol'] = eptm.edge_df['sub_vol'] * (
            1 - eptm.edge_df['is_basal'])
        eptm.cell_df['vol'] = eptm.sum_cell(eptm.edge_df['sub_vol'])


In [30]:
MixedGeometry.update_all(eptm)

In [ ]:

class MixedModel(SheetModel):
    
    @staticmethod
     def dimentionalize(mod_specs):
        """
        Changes the values of the input gamma and lambda parameters
        from the values of the prefered height and area.
        Computes the norm factor.
        """

        dim_mod_specs = deepcopy(mod_specs)

        Kv = dim_mod_specs['face']['vol_elasticity']
        A0 = dim_mod_specs['face']['prefered_area']
        h0 = dim_mod_specs['face']['prefered_height']
        gamma = dim_mod_specs['face']['contractility']

        dim_mod_specs['face']['contractility'] = gamma * Kv* A0 * h0**2
         dim_mod_specs['face']['prefered_vol'] = A0 * h0

        
        lbda = dim_mod_specs['edge']['line_tension']
        dim_mod_specs['edge']['line_tension'] = lbda * Kv * A0**1.5 * h0**2

        dim_mod_specs['settings']['grad_norm_factor'] = Kv * A0**1.5 * h0**2
        dim_mod_specs['settings']['nrj_norm_factor'] = Kv * (A0*h0)**2

        return dim_mod_specs
    

In [24]:
sheet_specs

{'edge': {'dx': 0.0,
  'dy': 0.0,
  'dz': 0.0,
  'face': 0,
  'length': 0.0,
  'nx': 0.0,
  'ny': 0.0,
  'nz': 1.0,
  'srce': 0,
  'trgt': 0},
 'face': {'area': 1.0,
  'is_alive': True,
  'num_sides': 6,
  'perimeter': 1.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0},
 'settings': {'geometry': 'cylindrical', 'height_axis': 'z'},
 'vert': {'basal_shift': 4.0,
  'is_active': True,
  'rho': 0.0,
  'x': 0.0,
  'y': 0.0,
  'z': 0.0}}